In [ ]:
import sys
import gplately
import numpy as np
import gplately.pygplates as pygplates
from gplately import ptt
import glob, os
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from plate_model_manager import PlateModelManager
import json

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
sys.path.append(ASPECT_LAB_DIR)
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')
import shilofue.GPlateLib as GPlateLib
# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities


### Define the function to calculate and filter subduction zone stats for South America

In [ ]:
# We are only interested in the latitude -55 and 5 in this notebook. 
# lat_samples = np.arange(-55,5,0.1)



def sample_subduction_center(one_subduction_data):
    '''
    '''
    one_subduction_data.arc_length
    pass

### Define the work flow to process and plot one trench

We select a trench by id, get the trench data from reconstruction, and resampled the trench

In [ ]:
# initiate the class
GClass = GPlateLib.GPLATE_CLASS()

In [ ]:
# assign a reconstruction time
reconstruction_time=0 # time of reconstruction, must be integar

GClass.SetReconstructionTime(reconstruction_time)
GClass.Reconstruct()

# assign the trench id for south america
trench_id = 201

one_subduction_data = GClass.GetOneSubductionByTrenchId(trench_id)

# plot the reconstructed zone
fig = plt.figure(figsize=(10,6), dpi=100)

ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 0))
gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])

# plot the coastline
GClass.PlotCoastlines(ax)

# plot the subduction zone
cb = GPlateLib.plot_one_subduction_data(ax, one_subduction_data)

cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)
plt.show()

In [ ]:
# assign a reconstruction time
reconstruction_time=0 # time of reconstruction, must be integar

GClass.SetReconstructionTime(reconstruction_time)
GClass.Reconstruct()

In [ ]:
# resample by a give arc length edge and resample section

arc_length_edge = 5.0  # by degree
arc_length_resample_section = 10.0

one_subduction_data_resampled, log_output_contents_temp= GClass.ResampleSubductionById(trench_id, arc_length_edge, arc_length_resample_section)
print(one_subduction_data_resampled)

# plot the reconstructed zone

fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 0))

gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])

# plot the coastline
GClass.PlotCoastlines(ax)

# plot the subduction zone
cb = GPlateLib.plot_one_subduction_data(ax, one_subduction_data_resampled)

cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)
plt.show()

In [ ]:
# initiate the class
GClass = GPlateLib.GPLATE_CLASS()

In [ ]:
# assign a reconstruction time
reconstruction_time=0 # time of reconstruction, must be integar

GClass.SetReconstructionTime(reconstruction_time)
GClass.Reconstruct()

In [ ]:
# resample by a give arc length edge and resample section
# todo_ages
arc_length_edge = 5.0  # by degree
arc_length_resample_section = 10.0
use_recorded_file = True; do_recording_file = False # use this option to start from a recorded file
# use_recorded_file = False; do_recording_file = True # use this option to start over

# resample the subduction zones
recorded_file = os.path.join(ASPECT_LAB_DIR, "files", "ThDSubduction", "gplate_json_files", "subduction_resampled_t_%.2e_edge_%.2f_section_%.2f.csv" \
                         % (reconstruction_time, arc_length_edge, arc_length_resample_section))
if use_recorded_file:
    assert(os.path.isfile(recorded_file))
    subduction_data_resampled = pd.read_csv(recorded_file) 
else:
    subduction_data_resampled = GClass.ResampleAllSubduction(arc_length_edge, arc_length_resample_section)
    subduction_data_resampled['age'] = [np.nan for i in range(len(subduction_data_resampled))]
    subduction_data_resampled['lon_fix'] = [np.nan for i in range(len(subduction_data_resampled))]
    subduction_data_resampled['lat_fix'] = [np.nan for i in range(len(subduction_data_resampled))]
    subduction_data_resampled['fix_age_polarity'] = [np.nan for i in range(len(subduction_data_resampled))]

# fix the ages
GClass.FixTrenchAge(subduction_data_resampled)
print(subduction_data_resampled.age)

In [ ]:
# get the invalid indexes
invalid_indexes = []
for i in range(len(subduction_data_resampled['age'])):
    if np.isnan(subduction_data_resampled['age'][i]):
        invalid_indexes.append(i)

print("len(ages): ")
print(len(subduction_data_resampled['age']))
print("ages: ")
print(subduction_data_resampled['age'])
print("invalid_indexes: ")
print(invalid_indexes)

# generate an output file for fixing the ages
fix_invalid_ds = [-1 for i in range(len(invalid_indexes))]
fix_invalid_thetas = [-1 for i in range(len(invalid_indexes))]
# todo_ages
temp_file = os.path.join(ASPECT_LAB_DIR, "dtemp", "gplate_export_test0", "subduction_resampled_t_%.2e_edge_%.2f_section_%.2f.csv" \
                         % (reconstruction_time, arc_length_edge, arc_length_resample_section))

subduction_data_resampled.to_csv(temp_file)
print("Data saved to %s" % temp_file)

In [ ]:
# basic plots
# plot the reconstructed zone
fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 180))
gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True
plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])
# plot the coastline
GClass.PlotCoastlines(ax)
# plot the seafloor age
im_age = GClass.PlotSeaFloorAges(ax)
# plot all subduction zones and subduction teeth
# im_sub = GPlateLib.plot_one_subduction_data(ax, GClass.GetSubductionData())
GClass.gplot.plot_trenches(ax, color='k')
GClass.gplot.plot_subduction_teeth(ax, color='k')

# plot all the fixed ages
mask = (~subduction_data_resampled['age'].isna())
ax.scatter(subduction_data_resampled[mask].lon, subduction_data_resampled[mask].lat, marker=".", s=60, c='r', transform=ccrs.PlateCarree())
ax.scatter(subduction_data_resampled[~mask].lon, subduction_data_resampled[~mask].lat, marker=".", s=60, c='y', transform=ccrs.PlateCarree())
ax.scatter(subduction_data_resampled[mask].lon_fix, subduction_data_resampled[mask].lat_fix, marker=".", s=30, c='c', transform=ccrs.PlateCarree())

# set plot options
cbar_age = plt.colorbar(im_age) # colorbar for ages
cbar_age.ax.get_yaxis().labelpad = 15
cbar_age.ax.set_ylabel("Age (Ma)", rotation=90)
# cbar = plt.colorbar(im_sub) # colorbar for trenches
cbar.ax.get_yaxis().labelpad = 15
# cbar.ax.set_ylabel('Trench Velocity Magnitude (in cm/yr)', rotation=90) # choose between these two labels to use for trenches
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)

# write outputs
fileout = os.path.join(RESULT_DIR, "gplate_subduction_zones", "subduction_resampled_t_%.2e_edge_%.2f_section_%.2f.pdf" \
                         % (reconstruction_time, arc_length_edge, arc_length_resample_section))
fileout1 = os.path.join(RESULT_DIR, "gplate_subduction_zones", "subduction_resampled_t_%.2e_edge_%.2f_section_%.2f.png" \
                         % (reconstruction_time, arc_length_edge, arc_length_resample_section))
if not (os.path.isdir(os.path.dirname(fileout))):
    os.mkdir(os.path.dirname(fileout))
fig.savefig(fileout)
fig.savefig(fileout1)
print("Save figure: %s" % fileout)
print("Save figure: %s" % fileout1)

In [ ]:
# todo_ages
# fix the invalid age manually
i = 1
theta = 182.0 # None: use the azimuth angle instead
d = 200e3 # m

# get the original pint
subduction_data_resampled_local = pd.DataFrame([subduction_data_resampled.iloc[i]])
# get the trench id
trench_id = subduction_data_resampled.iloc[i].trench_pid
if theta is None:
    theta = subduction_data_resampled.iloc[i].trench_azimuth_angle + 180.0

# migrate the point and interpolate age
subduction_data_resampled_local.lon, subduction_data_resampled_local.lat = \
    Utilities.map_point_by_distance(subduction_data_resampled.iloc[i].lon, subduction_data_resampled.iloc[i].lat, theta, d)
new_age = GClass.InterpolateAgeGrid(subduction_data_resampled_local)
print("theta: %.4f, d: %.4e" % (theta, d))
print("lon: %.4f, lat: %.4f" % (subduction_data_resampled_local.lon, subduction_data_resampled_local.lat))
print("new_age: ", new_age)

# get the data of this subduction zone
one_subduction_data = GClass.GetOneSubductionByTrenchId(trench_id)

# basic plots
# plot the reconstructed zone
fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 180))
gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True
plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])
# plot the coastline
GClass.PlotCoastlines(ax)
# plot the seafloor age
im_age = GClass.PlotSeaFloorAges(ax)

# plot the subduction zone
im_sub = GPlateLib.plot_one_subduction_data(ax, one_subduction_data)
# plot the original point
im_sub_point =ax.scatter(subduction_data_resampled.iloc[i].lon, subduction_data_resampled.iloc[i].lat, marker=".", s=60, c='r', transform=ccrs.PlateCarree())
# plot the migrated point
im_sub_point =ax.scatter(subduction_data_resampled_local.lon, subduction_data_resampled_local.lat, marker=".", s=30, c='c', transform=ccrs.PlateCarree())

# set plot options
cbar_age = plt.colorbar(im_age) # colorbar for ages
cbar_age.ax.get_yaxis().labelpad = 15
cbar_age.ax.set_ylabel("Age (Ma)", rotation=90)
cbar = plt.colorbar(im_sub) # colorbar for trenches
cbar.ax.get_yaxis().labelpad = 15
# cbar.ax.set_ylabel('Trench Velocity Magnitude (in cm/yr)', rotation=90) # choose between these two labels to use for trenches
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)

### Plot all trenches

We plot the convergence / trench retreat rate of trenches below.
The plot is also combined with a sea floor age

### Get the age of the subducting plate at these query points

Issue: We get a lot of nan value in ages
1. check the position of the sample points
2. plot the raster of oceanic plate ages

Here we generate a variation of the previous plot, by differentiating valid age values and invalid age values.

In [ ]:
# define all the markers to use

all_markers = ["o", "v", "^", "<", ">", "1", "2", "3", "4", 's', 'p', 'P', '*', 'h', 'x', 'D']

fig1, ax1 = plt.subplots()
total_points_plotted = 0 # record the number of plotted points

# sort out all the valid data
mask = (~subduction_data_resampled["age"].isna())
total_points_plotted += len(ages[mask])
subduction_data_resampled_valid = subduction_data_resampled[mask]

unique_trench_ids = subduction_data_resampled_valid.trench_pid.unique()
print(unique_trench_ids)
assert(len(unique_trench_ids) < len(all_markers))  # assert we have enough markers for trenches

i = 0
for trench_id in unique_trench_ids:
    mask1 = (subduction_data_resampled_valid.trench_pid == trench_id)
    ax1.plot(subduction_data_resampled_valid[mask1].age, subduction_data_resampled_valid[mask1].trench_velocity, ".", marker=all_markers[i], label="%d" % trench_id)
    i += 1

ax1.grid()
ax1.legend()

print("Total plotted points: %d" % total_points_plotted)

# configuration
ax1.set_xlim([0, 160.0])
ax1.set_ylim([-10.0, 10.0])
ax1.set_xlabel("Age (Ma)")
ax1.set_ylabel("Trench Velocity Magnitude (cm/yr)")

# ax2.plot(range(total_points_plotted), ages[mask], '*')